Script to detect flashbake blocks. Enter the block number on the last line then hit (Shift)+(Enter)

In [1]:
import requests
import json
import sys

def is_fb_block(level):
    ops = requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3?metadata=never").json()
    simplified_ops = []
    for op_num,op in enumerate(ops):
        sop = { "contents": []}
        for c in op["contents"]:
            sc = { "fee": c["fee"],
                    "gas_limit": c["gas_limit"]}
            sop["contents"].append(sc)
        sop["fee_sum"] = sum([int(c["fee"]) for c in op["contents"]])
        sop["operation_gas"] = sum([int(c["gas_limit"]) for c in op["contents"]])
        sop["gas_ratio"] = sop["operation_gas"] / 5200000
        # operation size in bytes. We perform a binary http query and substract 68 bytes metadata
        # NOPE ! 32 (protocol hash) + 32 (hash de l'op)  + 4 (chain id) => faut enlever 68 bytes
        # 32 (hash de l'op) + 4 (chain id) + 13 (??) => faut enlever 49 bytes
        extralen=49
        sop["operation_size"] = len(bytes(requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3/{op_num}?metadata=never", headers={'accept': 'application/octet-stream'}).content)) - extralen
        sop["size_ratio"] = sop["operation_size"] / (512*1024)
        sop["weight"] = sop["fee_sum"] / max(sop["size_ratio"], sop["gas_ratio"])
        sop["hash"] = op["hash"][0:6]
        simplified_ops.append(sop)
#print(json.dumps(simplified_ops, indent=2))
    if (sorted(simplified_ops, key=lambda op: op["weight"], reverse = True) == simplified_ops):
        print(f"Block {level} is not a FB block")
    else:
        print("operation_hash,operation_size,operation_gas,fee_sum,weight")
        for op in simplified_ops:
            print(f"{op['hash']},{op['operation_size']},{op['operation_gas']},{op['fee_sum']},{op['weight']}")
        print(f"Block {level} is a Flashbake block!")
        exit(1)

is_fb_block(2804310)

operation_hash,operation_size,operation_gas,fee_sum,weight
opLKrS,152,1001,0,0.0
opTQpy,11496,179100,49600,1440089.3355667223
op2p67,173,1495,461,1397091.1445086705
ood5a9,458,5965,1392,1213478.6253143335
oo431C,254,2984,691,1204155.4959785524
oofKQM,293,2264,656,1173832.518771331
oornHT,213,1308,453,1115035.042253521
ooUQzZ,270,3628,745,1067805.9536934951
oo9deK,458,5924,1174,1030519.9189736664
ooumso,153,10600,2000,981132.0754716981
op6MVW,356,9569,1650,896645.4174939911
oowKwa,1912,34405,5573,842307.8040982415
oosWyq,354,9563,1434,779755.3069120569
ooPoTH,352,9563,1432,778667.7820767541
ooWSvg,413,14308,1991,723595.191501258
ooywkh,155,10600,1420,696603.7735849057
onxp5t,181,12103,1528,656498.3888292159
ooFqxU,181,13225,1615,635009.4517958412
oosDGZ,181,14375,1755,634852.1739130435
oontNK,179,14551,1746,623957.1163493918
ongFvF,179,22399,2530,587347.6494486361
oos2rK,177,38914,4204,561772.1128642648
onvpTQ,189,46192,4919,553749.567024593
Block 2804310 is a Flashbake block!
